In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error as mae
df = pd.read_csv("/content/drive/MyDrive/BDM Project/Croston's Method.csv")

In [2]:
def alpha(demand):
    non_zero_demands= demand[demand>0] #Get the non zero demands from the data
    if len(non_zero_demands)<2:
        return 0.1 #if there are less than 2 non zero demands, we cannot optimize
    alphas = np.arange(0.1,1.0,0.1) #We will check from alpha = 0.1 to 0.9 with stepsize as 0.1
    best_alpha = 0.1 #We assume that best alpha is 0.1 as we will start iteration from here
    min_mae = float('inf') #We assume that minimum Mean Absolute Error is infinity
    #Now we loop through each alpha
    for current_alpha in alphas:
        forecasts = []
        smoothed_value = non_zero_demands.iloc[0] #Initialization of smoothing process
        for i in range(1,len(non_zero_demands)): # Generate forecasts for the historical non-zero demands using SES
            smoothed_value = current_alpha * non_zero_demands.iloc[i] + (1-current_alpha) * smoothed_value #Simple Exponential Smoothing (SES) formula
            forecasts.append(smoothed_value) #List of forecast for every alpha
        if len(forecasts)>0: #Calculating the MAE for this alpha's forecasts
            current_mae = mae(non_zero_demands.iloc[1:],forecasts)
            if current_mae < min_mae: #Finding the alpha having the least MAE
                min_mae = current_mae
                best_alpha = current_alpha
    return best_alpha

In [3]:
def croston_method(demand,alpha):
    demand_indices = np.flatnonzero(demand) #Finding all non-zero demands
    if len(demand_indices) == 0: #If there are no non-zero demands, the forecast is zero
        return 0
    z = demand[demand_indices] #The demand series (array of all the non-zero demands)
    p = np.diff(demand_indices,prepend=-1) #Interval series: difference between consecutive non-zero indices, with first interval = first nonzero position + 1.
    smoothed_z = np.zeros(len(z)) #Initialize two arrays to store the smoothed demands and intervals
    smoothed_p = np.zeros(len(p)) #Set the first smoothed values to be the first actual demand and interval
    smoothed_z[0] = z.iloc[0]
    smoothed_p[0] = p[0]
    for i in range(1,len(z)):
        smoothed_z[i] = alpha * z.iloc[i] + (1-alpha) * smoothed_z[i-1] #SES of demand size (z)
        smoothed_p[i] = alpha * p[i] + (1-alpha) * smoothed_p[i-1] #SES for demand interval (p)
    forecast = smoothed_z[-1]/smoothed_p[-1]
    return forecast

In [4]:
# Create an empty list to store the forecasts for each product.
forecasts = []

# Loop through each product column in the DataFrame, starting from the second column (index 1).
# We skip the first column because it contains the 'Month' data.
for i in range(1, len(df.columns)):
    # Get the column name for the current product.
    col_name = df.columns[i]

    # Extract the sales data for the current product.
    demand = df[col_name]

    # Call the 'alpha' function to find the best smoothing constant for this product's data.
    alpha_value = alpha(demand)

    # Call the 'croston_method' function to generate a forecast for the next period.
    # We pass in the demand data and the optimal alpha value.
    forecast = croston_method(demand, alpha_value)

    # Add the product's name and its forecast to the list in a formatted string.
    forecasts.append(f"{forecast.round()}")

# Print the final list of forecasted values for all products.
print("Forecasted Values:", forecasts)

Forecasted Values: ['2911.0', '2371.0', '619.0', '983.0', '271.0', '393.0', '479.0', '275.0', '162.0', '333.0', '291.0', '414.0', '338.0', '250.0', '162.0', '62.0', '49.0', '150.0', '229.0', '63.0', '60.0', '52.0', '50.0', '29.0', '39.0', '61.0', '34.0', '50.0', '25.0', '17.0', '26.0', '33.0', '20.0', '30.0', '17.0', '25.0', '17.0', '17.0', '16.0', '10.0', '3.0', '3.0', '7.0', '3.0', '10.0', '6.0', '5.0', '1.0', '1.0', '3.0', '5.0', '4.0', '1.0', '2.0', '1.0', '2.0', '2.0', '2.0', '2.0', '1.0', '1.0', '3.0', '2.0', '1.0', '1.0', '1.0', '1.0', '1.0', '0.0', '0.0', '1.0', '1.0', '2.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '1.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0']


Sources used for the code:
1) https://www.pmorgan.com.au/tutorials/error-metrics-for-intermittent-demand-cfe,-pis,-msr/
2) https://www.pmorgan.com.au/tutorials/crostons-method/